In [ ]:
!pip install bertopic nepalitokenizer nepali-stemmer pymongo snowballstemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bertopic-0.14.1-py2.py3-none-any.whl (120 kB)
  Using cached nepalitokenizer-1.8.6.0-py3-none-any.whl (8.2 kB)
  Using cached nepali_stemmer-0.0.2-py3-none-any.whl (149 kB)
  Using cached pymongo-4.3.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (492 kB)
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached sentencepiece-0.1.98-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub

In [ ]:
#Import the news headline in dataframe named news_text_df
import pymongo
import pandas as pd

myclient = pymongo.MongoClient("mongodburl")
mydb = myclient["major-project"]
mycol = mydb["news"]

mydoc = mycol.find({},{"Headline":1})

news_text_df = pd.DataFrame(list(mydoc))
news_text_df.head()

,_id,Headline
0,640751c6cd3e9c69b1269877,राष्ट्रपति निर्वाचनका लागि मतदान केन्द्र निर्माण
1,640751c6cd3e9c69b1269878,सभामुख घिमिरे बहराइन जाने
2,640751c6cd3e9c69b1269879,रवि लामिछानेको मुद्दामा फैसलाको पूर्णपाठले के ...
3,640751c6cd3e9c69b126987a,मुस्ताङमा बौद्ध विश्वविद्यालय खोल्न सरकारले स्...
4,640751c6cd3e9c69b126987b,रवि लामिछानेको राहदानी विवादबारे के बोल्यो सर्...


In [ ]:
#Import the news headline in dataframe named news_text_df
import pymongo
import pandas as pd

myclient = pymongo.MongoClient("mongodburl")
mydb = myclient["major-project"]
mycol = mydb["news_np"]

mydoc = mycol.find({},{"headline":1})

news_text_df = pd.DataFrame(list(mydoc))
news_text_df.head()

,_id,headline
0,6419e7569cd6f156db0ad4c6,"नेपालमा पश्चिमी वायुको प्रभाव, उच्च पहाडी क्षे..."
1,6419e7569cd6f156db0ad4c7,रेडक्रसको भ्रष्टाचारमा अन्देखा किन ?
2,6419e7569cd6f156db0ad4c8,तीन थान एयर गनसहित चार भारतीय नियन्त्रणमा
3,6419e7569cd6f156db0ad4c9,सांसद चौधरीको मुद्दामा उच्च अदालतको कारण देखाउ...
4,6419e7569cd6f156db0ad4ca,राष्ट्रपति पदमा गैरएमालेको खोजी : शेरबहादुर कु...


In [ ]:
docs=news_text_df["headline"]

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nepalitokenizer import NepaliTokenizer
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from umap import UMAP
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import snowballstemmer

stemmer = snowballstemmer.stemmer('nepali')
stopword= stopwords.words('nepali')

def nepali_tokenizer(text):
    # tokenize the text using the BERT tokenizer
    tokens = NepaliTokenizer().tokenizer(text)
    # Add stemming

    # return the token list as a string
    return stemmer.stemWords(tokens)

sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)
hdbscan_model = HDBSCAN(min_samples=100, gen_min_span_tree=True, prediction_data=True)

vectorizer = CountVectorizer(stop_words=stopword, tokenizer=nepali_tokenizer)

topic_model = BERTopic(embedding_model=sentence_model,
                       vectorizer_model=vectorizer,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       min_topic_size=100,
                       calculate_probabilities= False)
topics,probs = topic_model.fit_transform(docs,embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)


fig = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,879,-1_सुरू_तस्बिर_कांग्रेस_अष्ट्रेलिया
1,0,210,0_प्रधानमन्त्री_अवार्ड_काठमाडौं_महानगर
2,1,205,1_मान्_है_प्रिय_हुँ
3,2,174,2_रन_विकेट_विश्वकप_नेपाल
4,3,99,3_नेपाली_नेपाल_महोत्सव_विद्यार्
5,4,70,4_भिडिओ_सार्वजनिक_फिल्म_टिजर
6,5,59,5_करोड_कारोबार_लाख_हजार
7,6,56,6_पर्यटन_पर्यटक_पर्यटकीय_गन्तव्य
8,7,48,7_पक्राउ_आरोप_प्रहरी_हत्या
9,8,43,8_उडान_एयरलाइन्स_एयर_शुल्क


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1520,-1_भिडिओ_नेपाल_तस्बिर_अस्ट्रेलिया
1,0,232,0_काठमाडौं_अवार्ड_महानगर_प्रिमियर
2,1,224,1_मान्_खुसी_मन_मेरी
3,2,199,2_राष्ट्रपति_बैठक_दल_कांग्रेस
4,3,144,3_करोड_कारोबार_लाख_प्रतिशत
5,4,121,4_नेपाली_नेपाल_अष्ट्रेलिया_विद्यार्
6,5,102,5_नेपाल_रन_विश्वकप_विकेट
7,6,63,6_भिडिओ_फिल्म_सार्वजनिक_गीत


In [ ]:
topic_model.save("bert_m")

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,134570,-1_कांग्रेस_प्रधानमन्त्री_मृत्यु_ओली
1,0,10201,0_पक्राउ_प्रहरी_आरोप_तस्करी
2,1,4954,1_अग्रता_नेकपा_चितवन_निर्वाचित
3,2,4922,2_विद्यार्_विद्यालय_शिक्षक_परीक्षा
4,3,4572,3_आन्दोलन_प्रदर्शन_मुख्यमन्त्री_मधेस
...,...,...,...
244,243,5,243_सिनास_‘बटसावित्री’_तुख्यः_दीर्घायु
245,244,5,244_फैलिएला_पहेंलपुर_लहलह_मिर्चैया
246,245,5,245_ निर्वाचित_रैनादेवी_आँधीखोला_वटै
247,246,5,246_क्षत्रीय_जीतप्रकाश_सभापति_बैतडीबाट


In [ ]:
fig.write_html("/content/gdrive/MyDrive/major-project/bert_test.html")

In [ ]:
fig

In [ ]:
# Mounting Google Drive for fasttext file
from google.colab import drive

import pandas as pd

drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive

Mounted at /content/gdrive/
/content/gdrive/MyDrive
